In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/FinalProject_CV

/content/drive/My Drive/FinalProject_CV


In [ ]:
#!ls

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.device_count())
print(torch.cuda.is_available())

1.13.0+cu116
1
True


In [ ]:
# Importing libraries

import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

import numpy as np
import pandas as pd

from torch.utils import data
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

In [ ]:
# importing the zipfile module
from zipfile import ZipFile
  
# loading the temp.zip and creating a zip object
with ZipFile("./data/dataset.zip", 'r') as zObject:
  
    # Extracting all the members of the zip 
    # into a specific location.
    zObject.extractall(
        path="./data")

In [ ]:
# import os

# # folder path
# dir_path = r'./data/dataset/IMG'
# count = 0
# # Iterate directory
# for path in os.listdir(dir_path):
#     # check if current path is a file
#     if os.path.isfile(os.path.join(dir_path, path)):
#         count += 1
# print('File count:', count)

In [ ]:
dataroot = "./data/dataset/"
ckptroot = "./"

lr = 1e-4
weight_decay = 1e-5
batch_size = 32
num_workers = 8
test_size = 0.8
shuffle = True

epochs = 25
start_epoch = 0
resume = False

In [ ]:
# Helper functions

def toDevice(datas, device):
    """Enable cuda."""
    imgs, angles = datas
    return imgs.float().to(device), angles.float().to(device)


def augment(dataroot, imgName, angle):
    """Data augmentation."""
    name = dataroot + 'IMG/' + imgName.split('\\')[-1]
    current_image = cv2.imread(name)

    if current_image is None:
        print(name)

    current_image = current_image[65:-25, :, :]
    if np.random.rand() < 0.5:
        current_image = cv2.flip(current_image, 1)
        angle = angle * -1.0
    #print(current_image, angle)
    return current_image, angle

In [ ]:
# Data Loading
def load_data(data_dir, test_size):
    """Load training data and train validation split"""
    pass

    # reads CSV file into a single dataframe variable
    data_df = pd.read_csv(os.path.join(data_dir, 'driving_log.csv'),
                          names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])

    # Divide the data into training set and validation set
    train_len = int(test_size * data_df.shape[0])
    valid_len = data_df.shape[0] - train_len
    trainset, valset = data.random_split(
        data_df.values.tolist(), lengths=[train_len, valid_len])

    return trainset, valset

trainset, valset = load_data(dataroot, test_size)

In [ ]:

# Creating dataset

class TripletDataset(data.Dataset):

    def __init__(self, dataroot, samples, transform=None):
        self.samples = samples
        self.dataroot = dataroot
        self.transform = transform

    def __getitem__(self, index):
        batch_samples = self.samples[index]
        steering_angle = float(batch_samples[3])

        center_img, steering_angle_center = augment(self.dataroot, batch_samples[0], steering_angle)
        left_img, steering_angle_left     = augment(self.dataroot, batch_samples[1], steering_angle + 0.4)
        right_img, steering_angle_right   = augment(self.dataroot, batch_samples[2], steering_angle - 0.4)

        center_img = self.transform(center_img)
        left_img   = self.transform(left_img)
        right_img  = self.transform(right_img)
        
        center_img = center_img.transpose((2, 0, 1))
        left_img = left_img.transpose((2, 0, 1))
        right_img = right_img.transpose((2, 0, 1))

        #print(center_img.shape)

        return (center_img, steering_angle_center), (left_img, steering_angle_left), (right_img, steering_angle_right)

    def __len__(self):
        return len(self.samples)

In [ ]:
# Data Loader

print("==> Preparing dataset ...")
def data_loader(dataroot, trainset, valset, batch_size, shuffle, num_workers):
    """Self-Driving vehicles simulator dataset Loader.

    Args:
        trainset: training set
        valset: validation set
        batch_size: training set input batch size
        shuffle: whether shuffle during training process
        num_workers: number of workers in DataLoader

    Returns:
        trainloader (torch.utils.data.DataLoader): DataLoader for training set
        testloader (torch.utils.data.DataLoader): DataLoader for validation set
    """
    transformations = transforms.Compose(
        [transforms.Lambda(lambda x: (x / 127.5) - 1.0)])

    # Load training data and validation data
    training_set = TripletDataset(dataroot, trainset, transformations)
    trainloader = DataLoader(training_set,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             num_workers=num_workers)

    validation_set = TripletDataset(dataroot, valset, transformations)
    valloader = DataLoader(validation_set,
                           batch_size=batch_size,
                           shuffle=shuffle,
                           num_workers=num_workers)

    return trainloader, valloader


trainloader, validationloader = data_loader(dataroot,
                                            trainset, valset,
                                            batch_size,
                                            shuffle,
                                            num_workers)

==> Preparing dataset ...


In [ ]:
# Creating model

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # Input image size = 70x320x3

        self.conv1 = nn.Conv2d(3, 16, 3)  # 68x318
        self.conv2 = nn.Conv2d(16, 32, 3) # 66x316
        self.pool1 = nn.MaxPool2d(2, 2) # 33x158

        self.conv3 = nn.Conv2d(32, 64, 3) # 31x156
        self.conv4 = nn.Conv2d(64, 64, 3) # 29x154
        self.pool2 = nn.MaxPool2d(2, 2) # 14x77

        self.conv5 = nn.Conv2d(64, 64, 3) # 12x75
        self.conv6 = nn.Conv2d(64, 64, 3) # 10x73
        self.conv7 = nn.Conv2d(64, 64, 3) # 8x71
        self.pool3 = nn.MaxPool2d(2, 2) # 4x35

        
        self.fc1 = nn.Linear(4*35*64, 128)
        self.dropout1 = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc3 = nn.Linear(64, 1)
        

    def forward(self, x):
  
        x = F.relu(self.conv1(x))
        x = self.pool1(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool2(F.relu(self.conv4(x)))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool3(F.relu(self.conv7(x)))

        x = x.view(-1, 4*35*64)
        x = F.relu(self.fc1(x))
        #print(x.shape)
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# Define model
print("==> Initialize model ...")
model = ConvNet()
print("==> Initialize model done ...")

==> Initialize model ...
==> Initialize model done ...


In [ ]:
# Define optimizer and criterion
optimizer = optim.Adam(model.parameters(),
                       lr=lr,
                       weight_decay=weight_decay)
criterion = nn.MSELoss()

In [ ]:
# learning rate scheduler
scheduler = MultiStepLR(optimizer, milestones=[30, 50], gamma=0.1)

# transfer to gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# Resuming training

if resume:
    print("==> Loading checkpoint ...")
    checkpoint = torch.load("../input/pretrainedmodels/model2.h5",
                            map_location=lambda storage, loc: storage)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    scheduler.load_state_dict(checkpoint['scheduler'])

In [ ]:
# Model training

class Trainer(object):
    """Trainer."""

    def __init__(self,
                 ckptroot,
                 model,
                 device,
                 epochs,
                 criterion,
                 optimizer,
                 scheduler,
                 start_epoch,
                 trainloader,
                 validationloader):
        """Self-Driving car Trainer.

        Args:
            model:
            device:
            epochs:
            criterion:
            optimizer:
            start_epoch:
            trainloader:
            validationloader:

        """
        super(Trainer, self).__init__()

        self.model = model
        self.device = device
        self.epochs = epochs
        self.ckptroot = ckptroot
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.start_epoch = start_epoch
        self.trainloader = trainloader
        self.validationloader = validationloader

    def train(self):
        """Training process."""
        self.model.to(self.device)
        Training_loss = []
        Validation_loss = []
        epochs = []
        actuals = []
        for epoch in range(self.start_epoch, self.epochs + self.start_epoch):
            epochs.append(epoch)
            self.scheduler.step()
            
            # Training
            train_loss = 0.0
            self.model.train()

            for local_batch, (centers, lefts, rights) in enumerate(self.trainloader):
                # Transfer to GPU
                centers, lefts, rights = toDevice(centers, self.device), toDevice(
                    lefts, self.device), toDevice(rights, self.device)

                # Model computations
                self.optimizer.zero_grad()
                datas = [centers, lefts, rights]
                for data in datas:
                    imgs, angles = data
                    # print("training image: ", imgs.shape)
                    outputs = self.model(imgs)
                    loss = self.criterion(outputs, angles.unsqueeze(1))
                    loss.backward()
                    self.optimizer.step()

                    train_loss += loss.data.item()

                if local_batch % 100 == 0:

                    print("Training Epoch: {} | Loss: {}".format(epoch, train_loss / (local_batch + 1)))
                    Training_loss.append(train_loss)


            # Validation
            self.model.eval()
            valid_loss = 0
            with torch.set_grad_enabled(False):
                for local_batch, (centers, lefts, rights) in enumerate(self.validationloader):
                    # Transfer to GPU
                    centers, lefts, rights = toDevice(centers, self.device), toDevice(
                        lefts, self.device), toDevice(rights, self.device)

                    # Model computations
                    self.optimizer.zero_grad()
                    datas = [centers, lefts, rights]
                    predicts = []
                    for data in datas:
                        imgs, angles = data
                        actuals.append(angles)
                        outputs = self.model(imgs)
                        loss = self.criterion(outputs, angles.unsqueeze(1))
                        predicts.append(outputs)

                        valid_loss += loss.data.item()

                    if local_batch % 100 == 0:
                        print("Validation Loss: {}".format(valid_loss / (local_batch + 1)))
                        Validation_loss.append(valid_loss)
                        

            #plt.plot(Validation_loss,epochs)
            #print('MSE:',mean_squared_error(actuals,predicts))
            print()
            # Save model
            if epoch % 5 == 0 or epoch == self.epochs + self.start_epoch - 1:

                state = {
                    'epoch': epoch + 1,
                    'state_dict': self.model.state_dict(),
                    'optimizer': self.optimizer.state_dict(),
                    'scheduler': self.scheduler.state_dict(),
                }

                self.save_checkpoint(state)

    def save_checkpoint(self, state):
        """Save checkpoint."""
        print("==> Save checkpoint ...")
        if not os.path.exists(self.ckptroot):
            os.makedirs(self.ckptroot)

        torch.save(state, self.ckptroot + 'model2-{}.h5'.format(state['epoch']))

In [ ]:
print("==> Start training ...")
trainer = Trainer(ckptroot,
                  model,
                  device,
                  epochs,
                  criterion,
                  optimizer,
                  scheduler,
                  start_epoch,
                  trainloader,
                  validationloader)
trainer.train()

==> Start training ...
Training Epoch: 0 | Loss: 0.7383577525615692
Training Epoch: 0 | Loss: 0.7006666230240671
Training Epoch: 0 | Loss: 0.6592538889997931
Training Epoch: 0 | Loss: 0.6318877640975472
Training Epoch: 0 | Loss: 0.6105410110790682
Training Epoch: 0 | Loss: 0.5972550029303142
Training Epoch: 0 | Loss: 0.5854216708047021
Training Epoch: 0 | Loss: 0.57555641507605
Training Epoch: 0 | Loss: 0.5668152630347735
Validation Loss: 0.4354540556669235
Validation Loss: 0.4835563448100987
Validation Loss: 0.4901327327523955

==> Save checkpoint ...
Training Epoch: 1 | Loss: 0.6394856870174408
Training Epoch: 1 | Loss: 0.4971366111779272
Training Epoch: 1 | Loss: 0.5083160162694863
Training Epoch: 1 | Loss: 0.5086618172456556
Training Epoch: 1 | Loss: 0.504004492245596
Training Epoch: 1 | Loss: 0.49856710061059145
Training Epoch: 1 | Loss: 0.49713921415838047
Training Epoch: 1 | Loss: 0.4962020101325956
Training Epoch: 1 | Loss: 0.49274056019045215
Validation Loss: 0.465339474380016

In [ ]:
state = {
        'model': model.module if device == 'cuda' else model,
        }

torch.save(state, 'model2.h5')